# Lab 06b
## Logit and Machine Learning

Let's turn to logit models. These models have been used in a variety of settings and fields, but are often called **classifiers** in a machine learning context. For logit models, the dependendent (i.e. Y variable or LHS variable) is a **binary** outcome, or 0/1 value. So, did you do something or not? Can we classify this observation? 

Logits are again covered in **Hull, Chapters 3 and 10**. 

Logit models are also covered in [Chapter 13 of our notes](https://aaiken1.github.io/fin-data-analysis-text/chapters/13_logit_credit.html). I work through the Hull examples there.

By the way, you can also use what are called [multi-nomial logit models](https://en.wikipedia.org/wiki/Multinomial_logistic_regression) for non-binary outcomes. 

You're going to see **more math now**, especially if you want to understand more about what's going on under-the-hood, so to speak. Do you understand what these different models are trying to do? Why we use them?

Use our online notes to complete our labs. I have my own commentary and links that will help. You can work through the parts below in order.

If you are getting errors and are not sure why, my first suggestion is always to **Restart** the Python kernel, to **Clear All Output**, and run each code cell **one-by-one** from the top.

Let each code cell represent one idea, or output. Take advantage of **Markdown** in your write-up. Use headers and formatting. Here's a [Markdown cheat sheet](https://www.markdownguide.org/cheat-sheet/) that I keep handy. Once you figure out some of the basics, you might not want to go back to Word or Google Docs. 

## Part 01

Like in Lab 06a, we are going to work through a problem from the Hull textbook. 

The full data set for LendingClub can be found on our Github page: <https://github.com/aaiken1/fin-data-analysis-python/raw/main/data/lending_clubFull_Data_Set.xlsx>. 

You are trying to predict **loan status**. You should extend the model from Chapter 3 to include additional features, such as the loan amount, term, interest rate, grade, verification status, and purpose of loan. Use regularization as appropriate. Report on the performance of the model you consider to be best. Finally, how might Lending Tree use this data and models to set interest rates?

Seems easy, right?

Some tips:

- **Pay attention to your workflow**. Doing things out of order is going to create major headaches. These tips follow my workflow and trouble I ran into.

- You're again bringing in an Excel file, so you might need to specify exactly how you need to do that. 

- I look at my data **a lot**. I use `lt.columns` a lot, where `lt` is what I called my data frame. 

- **Pay attention to data types**. Print out all of your variables and take a look. Are your numeric variables numeric? Do you need to remove any text from some variables? Need to fix any missing values? Or just drop those rows? What are the steps to go from text variables to indicators using `pd.get_dummies()`? You want 1/0 variables to go into your models - True and False might not work. Does **loan status** itself need some work? I created my own loan status variable using the one in the data. You'll have to make some choices about the data. Make sure that you understand what each variable is measuring.

- OK, one last time. Print all of your variables names. Use `.describe()`. Look for missings. **Only move on if you have what you want.**

- Standardize your variables **only** after you have cleaned your features and only included what you need in the **X** data frame. I used the `.StandardScaler()' method, rather than do things "by hand", like in the Hull book.

- Split your data into **training** and **testing** sub-sets, but, again, **only after everything is cleaned up**. You can also do **training**, **validation**, and **testing**, like the text. See the notes for more. The general idea is the same, regardless. 

- You are going to fit your model using the testing data. I started by using `smf.logit`, rather than `sklearn`. I had to combine my **y_train** and **X_train** dataframes, since `smf.logit` wants everything in the same place (or, at least its easier that way). I then specified my **formula**, where my new loan_status variable was my y variable, the thing I'm trying to predict. To start, I used just annual income, dit, FICO, loan amount, term, and interest rate as my X, or independent, or explanatory, variables. See the notes for more.

- Why use `smf.logit`, rather than `sklearn`, like in the Hull book? I like the output. `sklearn` isn't concerned with the coefficients, t-stats, etc. I like to see the model output, if I can, especially I have some idea what the variables mean and what they might predict. Even when doing machine learning, I think that **econnomic interpretation** is important. For example, is the sign on annual income "correct"? 

- Now, try using `smf.logit` and include all of your `grade_` indicators. So, `grade_A`, `grade_B`, etc. Print the summary. Does anything look off to you?

- See all of those *nan* values? This model didn't actually run. Why? We included an **intercept** and all of the indicators/dummies for **grade**. This is an example of the **dummy variable trap** discussed in Hull, Chapter 3 and the notes. From Chat GPT:

> The dummy variable trap, also known as the dummy variable multicollinearity trap, occurs in regression analysis when categorical variables are represented as dummy variables (0 or 1) in a regression model, and one dummy variable can be predicted from the others. For example, consider a categorical variable "color" with three categories: red, blue, and green. To include this variable in a regression model, you might create two dummy variables: "is_blue" and "is_green". If you include all three dummy variables (including "is_red") in the model, it would create perfect multicollinearity because "is_red" can always be inferred from "is_blue" and "is_green" (if both are 0, then it must be red).

- `pd.get_dummies` is using a method called **one-hot encoding**. Every value in a column gets "pulled out" and mode its own column, with values of *True* or *False*. We then convert these to 1/0. The problem comes when [inverting the matrix when estimating the model](https://en.wikipedia.org/wiki/Multicollinearity). You can't do that when some combination of columns perfectly predicts another, as described above. How can you fix this? You should **only include k-1 of your indicators**, where k is the total number of indicator variables for that category (e.g. color above). Then, everything is relative to the one group that you left out, if you're trying to interpret your model. If you're just predicting, don't worry about it.

- Keep using `smf.logit` to run your model and print the summary. But, now, include all of the dummy variables, excluding one from each category. See how your formula is getting really long, because you have so many features? You can split a formula across multiple lines using a single `\` like:

```
formula = 'loan_status_ind ~ annual_inc + dti + fico_range_low + loan_amnt + term + int_rate + \
            grade_B + grade_C + grade_D + grade_E + grade_F + grade_G + \
```

- **Let's move to the `sklearn` method, like in the Hull book**. Take the **X_train** data frame and **drop the variables that you don't want to include**. I would save this as a new training set. Note that sometimes its easier to drop columns, while other times it's easier to use `.loc` and keep columns. 

- Try doing a `sklearn` logit with just the numeric variable. Print the coefficients. Add those grade indicators. Check and see if your answer is close to what you got with the `smf.logit` method. 

- Try the logit with everything you included in the final `smf.logit` model. To get mine to run, I needed to change my method and the max number of iterations. The logit model is "searching" for coefficients to solve the model. There are various ways to go about this and you can change how long it tries. Add this to code where you define the `LogisticRegression` object:

```
solver="sag", max_iter = 5000
```

- Between the two methods, my coefficients were close, but not exactly the same. Estimating logits gets "tricky" when you have so many features included in the model and multicollinearity is a problem, even after fixing the dummy variables. You can actually run LASSO and Ridge models with logit using `sklearn`. By using **regularization**, we can help mitigate the "too many features" issue, just like with linear regression. You'll notice that we haven't discussed **hyperparameters** here - that's because a regular logit classifier doesn't have them! However, the training-testing split is still useful, as you're about to see. 

- Once you've estimated the model with all of your features, let's try some **prediction**. Follow along with the Hull text and online notes to generate **predicted values** using your training data and `lgstc_reg.predict_proba()`. Turn the probabilities into binary predictions, like in the notes. You'll need to pick a **threshold** to do this. Your threshold (e.g. 0.85) turns a probability into a 1/0 prediction. If the predicted probability is above the threshold, you get a 1, else 0. Create a **confusion matrix** for a variety of thresholds. See the notes and the Hull book for more on confusion matrices. You're trying to get a sense for the number predictive power and the number of mistakes made by your model. 

- Finally, once you have a threshold that you like, use your model with the **X_test** features that you held out to predict loan status. Compare these predictions with the **y_test** hold out sample. Use your threshold from the training step above to create binary predictors and another confusion matrix. 

- I like to look at the mean value of the predicted values compared to the mean values of the actual values. For example, if my loan status is actually good 90% of the time, is the average of my predicted values also around 90%? Just a quick eyeball test to help with my thresholds. **The book and notes give you better ways to evaluate thresholds**. The threshold is kind of like our hyperparater here.

- What do you think? Is predicting loan default easy or difficult? How might you use this model if you were at FinTech doing loan underwriting? What trade-offs do you face?

In [190]:
# import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Include this to have plots show up in your Jupyter notebook.
%matplotlib inline 

# For the Hull logit
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, auc, average_precision_score

from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', 40)

In [191]:
url = 'https://github.com/aaiken1/fin-data-analysis-python/raw/main/data/lending_clubFull_Data_Set.xlsx'
lt = pd.read_excel(url, engine='openpyxl')

I put all of my import statements at the top. That way, I don't import things multiple times. This can lead to issues in more complicated work, where you might be importing specific versions of libraries.

I then bring in the Excel file using the `openpyxl` engine. You probably needed to do a `pip install` to get that into your Codespace.

Now, let's look at the data types. Remember, if you see *object*, this is they way that `pandas` deals with data types that it doesn't really understand. Typically, text for us.

In [192]:
lt.dtypes

Unnamed: 0                        int64
id                                int64
member_id                        object
loan_amnt                       float64
term                             object
                              ...      
settlement_status                object
settlement_date          datetime64[ns]
settlement_amount               float64
settlement_percentage           float64
settlement_term                 float64
Length: 135, dtype: object

I start by **just keeping the columns that I want**. This way, I'm not keeping a bunch of features in memory and wasting compute time.

In [193]:
lt = lt.loc[:, ['id', 'member_id', 'annual_inc', 'dti', 'fico_range_low', 'loan_amnt', 'term', 'int_rate', 'grade', 'home_ownership', 'purpose', 'verification_status', 'loan_status']]

In [194]:
lt.head()

,id,member_id,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,grade,home_ownership,purpose,verification_status,loan_status
0,263591,545710,44304.0,18.47,690.0,20000.0,60 months,17.93,E,MORTGAGE,debt_consolidation,Verified,Charged Off
1,1613916,69664096,136000.0,20.63,670.0,30000.0,36 months,11.99,C,MORTGAGE,debt_consolidation,Verified,Current
2,818934,8965180,50000.0,29.62,735.0,21500.0,36 months,11.99,B,RENT,debt_consolidation,Source Verified,Fully Paid
3,1606612,70572960,64400.0,16.68,675.0,10000.0,36 months,13.67,C,RENT,debt_consolidation,Source Verified,Fully Paid
4,1639932,68589517,88000.0,5.32,660.0,5000.0,36 months,8.49,B,MORTGAGE,debt_consolidation,Source Verified,Current


I noticed the text in *term* right away. Let's get rid of that and make the months numeric.

In [195]:
lt['term'] = pd.to_numeric(lt['term'].str.replace(' months', ''))

In [196]:
lt.head()

,id,member_id,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,grade,home_ownership,purpose,verification_status,loan_status
0,263591,545710,44304.0,18.47,690.0,20000.0,60.0,17.93,E,MORTGAGE,debt_consolidation,Verified,Charged Off
1,1613916,69664096,136000.0,20.63,670.0,30000.0,36.0,11.99,C,MORTGAGE,debt_consolidation,Verified,Current
2,818934,8965180,50000.0,29.62,735.0,21500.0,36.0,11.99,B,RENT,debt_consolidation,Source Verified,Fully Paid
3,1606612,70572960,64400.0,16.68,675.0,10000.0,36.0,13.67,C,RENT,debt_consolidation,Source Verified,Fully Paid
4,1639932,68589517,88000.0,5.32,660.0,5000.0,36.0,8.49,B,MORTGAGE,debt_consolidation,Source Verified,Current


Now, I'm going to look at these different **categorical variables**. What values can they take? I want to know what my eventual dummy variables are going to look like.

In [197]:
lt['grade'].unique()

array(['E', 'C', 'B', 'F', 'D', 'A', 'G', nan], dtype=object)

In [198]:
lt['home_ownership'].unique()

array(['MORTGAGE', 'RENT', 'OWN', 'ANY', 'OTHER', 'NONE', nan],
      dtype=object)

In [199]:
lt['purpose'].unique()

array(['debt_consolidation', 'credit_card', 'small_business', 'medical',
       'other', 'vacation', 'house', 'major_purchase', 'home_improvement',
       'wedding', 'car', 'moving', 'renewable_energy', 'educational', nan],
      dtype=object)

In [200]:
lt['verification_status'].unique()

array(['Verified', 'Source Verified', 'Not Verified', nan], dtype=object)

In [201]:
lt['loan_status'].unique()

array(['Charged Off', 'Current', 'Fully Paid', 'Late (31-120 days)',
       'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Default',
       nan], dtype=object)

OK, *loan_status* is my **target variable**. This is what I'm trying to predict. In the original Hull example, he'd alrady cleaned up the variable. Here, we need to make some choices. I'm going to combine a few of these categories and create my own **indicator variable**. 

I'll define *loan_status_ind == 1* as a loan in good standing. You could absolutely define this the other way around. 

In [202]:
lt['loan_status_ind'] = np.where((lt['loan_status'] == 'Current') | (lt['loan_status'] == 'Fully Paid'), 1, 0)

In [203]:
lt['loan_status_ind'].describe()

count    25000.000000
mean         0.871840
std          0.334275
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: loan_status_ind, dtype: float64

Using my definition, about 87% of the loans are in good standing. 

Now, let's deal with missing values.

In [204]:
missing_values = lt.isnull().sum()
missing_table = pd.DataFrame({'Variable': missing_values.index, 'Missing Values': missing_values.values})
missing_table


,Variable,Missing Values
0,id,0
1,member_id,0
2,annual_inc,1
3,dti,12
4,fico_range_low,1
5,loan_amnt,1
6,term,1
7,int_rate,1
8,grade,1
9,home_ownership,1


Not too many. I used `.isnull` to "tag" missing observations. There's also `.isna`. The difference, according to Claude:

In pandas, .isnull() and .isna() are both methods used to detect missing or null values in a pandas DataFrame or Series. However, there is a subtle difference between the two:

- .isnull(): This method is the older and more traditional way of detecting null values in pandas. It checks for both None and np.nan (Not a Number) values. It is an alias for the notnull method with the boolean values inverted.

- .isna(): This method is the newer and recommended way of detecting null values in pandas. It checks for both None and np.nan values, as well as other types of null values that may be present in the data (e.g., pd.NA for the new pandas extension arrays). It is an alias for the notna method with the boolean values inverted.

In most cases, .isna() and .isnull() will produce the same results when working with traditional NumPy data types. However, .isna() is more general and should be preferred because it can correctly handle missing values in the newer pandas extension data types, which .isnull() may not handle correctly.

In [205]:
missing_counts = lt.isna().sum()
missing_table = pd.DataFrame({'Variable': missing_counts.index, 'Missing Values': missing_counts.values})
missing_table


,Variable,Missing Values
0,id,0
1,member_id,0
2,annual_inc,1
3,dti,12
4,fico_range_low,1
5,loan_amnt,1
6,term,1
7,int_rate,1
8,grade,1
9,home_ownership,1


With so few missing values, I'm not going to worry about imputing anything. Let's just drop an observation if any of the features are missing.

If I was being really careful here, I maybe wouldn't drop an observation if *id* or *member_id* were missing, since these aren't technically features that we're using. 

In [206]:
lt.dropna(inplace=True)

missing_counts = lt.isna().sum()
missing_table = pd.DataFrame({'Variable': missing_counts.index, 'Missing Values': missing_counts.values})
missing_table

,Variable,Missing Values
0,id,0
1,member_id,0
2,annual_inc,0
3,dti,0
4,fico_range_low,0
5,loan_amnt,0
6,term,0
7,int_rate,0
8,grade,0
9,home_ownership,0


OK, looks good - no more missing values. Remember, we need to get rid of all missing values somehow if we want our models in `sklearn` to run. Other tools, like `statsmodel` might take a data frame with missing values, but then just drop them for you. 

Either way, missing values can't be part of your model.

Now, let's use `pd.get_dummies` to create some 1/0 indicator variables. The last line of code drops the original variables. This is **really important**. The original variables are text and can't be used as features, at least not using our basic set-up and tools. 

In [207]:
# Create indicator variables using pd.get_dummies()
grade_dummies = pd.get_dummies(lt['grade'], prefix='grade')
grade_dummies = grade_dummies.astype(int)

home_ownership_dummies = pd.get_dummies(lt['home_ownership'], prefix='home_ownership')
home_ownership_dummies = home_ownership_dummies.astype(int)

purpose_dummies = pd.get_dummies(lt['purpose'], prefix='purpose')
purpose_dummies = purpose_dummies.astype(int)

verification_status_dummies = pd.get_dummies(lt['verification_status'], prefix='verification_status')
verification_status_dummies = verification_status_dummies.astype(int)

# Add the indicator variables to the lt dataframe
lt = pd.concat([lt, grade_dummies, home_ownership_dummies, purpose_dummies, verification_status_dummies], axis=1)

lt.drop(['grade', 'home_ownership', 'purpose', 'verification_status', 'loan_status'], axis=1, inplace=True)

In [208]:
lt.head()

,id,member_id,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,loan_status_ind,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,263591,545710,44304.0,18.47,690.0,20000.0,60.0,17.93,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1613916,69664096,136000.0,20.63,670.0,30000.0,36.0,11.99,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,818934,8965180,50000.0,29.62,735.0,21500.0,36.0,11.99,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1606612,70572960,64400.0,16.68,675.0,10000.0,36.0,13.67,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1639932,68589517,88000.0,5.32,660.0,5000.0,36.0,8.49,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [209]:
lt.describe()

,id,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,loan_status_ind,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
count,2.498800e+04,2.498800e+04,24988.00000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000
mean,8.260204e+05,7.710003e+04,18.61075,695.574276,14686.635585,42.848087,13.217519,0.871858,0.165359,0.298103,0.293701,0.141828,0.071474,0.023491,0.006043,0.000120,0.494277,0.000080,0.000160,0.106931,0.398431,0.010925,0.218865,0.582199,0.000040,0.064511,0.004162,0.021210,0.011365,0.006843,0.058428,0.000440,0.011606,0.008124,0.001281,0.314711,0.379342,0.305947
std,4.788446e+05,5.495843e+04,14.04028,31.260142,8764.135109,10.838012,4.740655,0.334254,0.371512,0.457434,0.455466,0.348881,0.257621,0.151461,0.077502,0.010957,0.499977,0.008946,0.012651,0.309032,0.489585,0.103953,0.413485,0.493207,0.006326,0.245666,0.064380,0.144087,0.106003,0.082442,0.234556,0.020977,0.107104,0.089768,0.035763,0.464410,0.485233,0.460817
min,6.000000e+00,2.000000e+02,0.00000,640.000000,600.000000,36.000000,5.320000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.084595e+05,4.600000e+04,12.06000,670.000000,8000.000000,36.000000,9.760000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.241515e+05,6.500000e+04,17.81000,690.000000,12712.500000,36.000000,12.740000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.243221e+06,9.200000e+04,24.27000,710.000000,20000.000000,60.000000,15.990000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,1.646774e+06,1.500000e+06,999.00000,845.000000,40000.000000,60.000000,30.990000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


I think I want to combine those last two verification status columns into a single indicator.

In [210]:
lt['verification_status_Verified'] = np.where(lt['verification_status_Source Verified'] == 1, 1, lt['verification_status_Verified'])


In [211]:
lt.drop("verification_status_Source Verified", axis=1, inplace=True)


In [212]:
lt.head()

,id,member_id,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,loan_status_ind,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,verification_status_Not Verified,verification_status_Verified
0,263591,545710,44304.0,18.47,690.0,20000.0,60.0,17.93,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1613916,69664096,136000.0,20.63,670.0,30000.0,36.0,11.99,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,818934,8965180,50000.0,29.62,735.0,21500.0,36.0,11.99,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1606612,70572960,64400.0,16.68,675.0,10000.0,36.0,13.67,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1639932,68589517,88000.0,5.32,660.0,5000.0,36.0,8.49,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [213]:
lt.describe()

,id,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,loan_status_ind,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,verification_status_Not Verified,verification_status_Verified
count,2.498800e+04,2.498800e+04,24988.00000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000
mean,8.260204e+05,7.710003e+04,18.61075,695.574276,14686.635585,42.848087,13.217519,0.871858,0.165359,0.298103,0.293701,0.141828,0.071474,0.023491,0.006043,0.000120,0.494277,0.000080,0.000160,0.106931,0.398431,0.010925,0.218865,0.582199,0.000040,0.064511,0.004162,0.021210,0.011365,0.006843,0.058428,0.000440,0.011606,0.008124,0.001281,0.314711,0.685289
std,4.788446e+05,5.495843e+04,14.04028,31.260142,8764.135109,10.838012,4.740655,0.334254,0.371512,0.457434,0.455466,0.348881,0.257621,0.151461,0.077502,0.010957,0.499977,0.008946,0.012651,0.309032,0.489585,0.103953,0.413485,0.493207,0.006326,0.245666,0.064380,0.144087,0.106003,0.082442,0.234556,0.020977,0.107104,0.089768,0.035763,0.464410,0.464410
min,6.000000e+00,2.000000e+02,0.00000,640.000000,600.000000,36.000000,5.320000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.084595e+05,4.600000e+04,12.06000,670.000000,8000.000000,36.000000,9.760000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.241515e+05,6.500000e+04,17.81000,690.000000,12712.500000,36.000000,12.740000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.243221e+06,9.200000e+04,24.27000,710.000000,20000.000000,60.000000,15.990000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,1.646774e+06,1.500000e+06,999.00000,845.000000,40000.000000,60.000000,30.990000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


I'm looking at these indicators. Many of them have very few observations. This is going to make estimating the logits below more difficult. I'm going to:

- combine *grade_E*, *grade_F*, and *grade_G*
- create just three *home_ownership* indicators: mortgage, rent, and not either. Own is when you own outright, with no mortage, I would think.
- create just three *purpose* indicators: credit care, debt consolidation, and other

How do I know to do this? I don't know. 25+ years experience? May not even be the best thing to do. But, these are the types of choices you try and help to illustrate **why it is so important to look at your data**.

This is also a great step to have Copilot write the code for me. My prompt:

> Create a new grade_E_below indicator that is the sum of grade_E, grade_F, and grade_G. Then, drop grade_E, grade_F, and grade_G from the lt dataframe.


In [214]:
lt['grade_E_below'] = lt['grade_E'] + lt['grade_F'] + lt['grade_G']

lt.drop(['grade_E', 'grade_F', 'grade_G'], axis=1, inplace=True)

In [215]:
lt['home_ownership_not_mortgage_rent'] = lt['home_ownership_ANY'] + lt['home_ownership_NONE'] + lt['home_ownership_OTHER'] + lt['home_ownership_OWN']

lt.drop(['home_ownership_ANY', 'home_ownership_NONE', 'home_ownership_OTHER', 'home_ownership_OWN'], axis=1, inplace=True)

In [216]:
lt['purpose_other_reasons'] = lt['purpose_car'] + lt['purpose_educational'] + lt['purpose_home_improvement'] + lt['purpose_house'] + lt['purpose_major_purchase'] + lt['purpose_medical'] + lt['purpose_moving'] + lt['purpose_other'] + lt['purpose_renewable_energy'] + lt['purpose_small_business'] +lt['purpose_vacation'] + lt['purpose_wedding']

lt = lt.drop(['purpose_car', 'purpose_educational', 'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase', 'purpose_medical', 'purpose_moving', 'purpose_other', 'purpose_renewable_energy', 'purpose_small_business', 'purpose_vacation', 'purpose_wedding'], axis=1)

In [217]:
lt.describe()

,id,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,loan_status_ind,grade_A,grade_B,grade_C,grade_D,home_ownership_MORTGAGE,home_ownership_RENT,purpose_credit_card,purpose_debt_consolidation,verification_status_Not Verified,verification_status_Verified,grade_E_below,home_ownership_not_mortgage_rent,purpose_other_reasons
count,2.498800e+04,2.498800e+04,24988.00000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000,24988.000000
mean,8.260204e+05,7.710003e+04,18.61075,695.574276,14686.635585,42.848087,13.217519,0.871858,0.165359,0.298103,0.293701,0.141828,0.494277,0.398431,0.218865,0.582199,0.314711,0.685289,0.101008,0.107291,0.198935
std,4.788446e+05,5.495843e+04,14.04028,31.260142,8764.135109,10.838012,4.740655,0.334254,0.371512,0.457434,0.455466,0.348881,0.499977,0.489585,0.413485,0.493207,0.464410,0.464410,0.301346,0.309490,0.399207
min,6.000000e+00,2.000000e+02,0.00000,640.000000,600.000000,36.000000,5.320000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.084595e+05,4.600000e+04,12.06000,670.000000,8000.000000,36.000000,9.760000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.241515e+05,6.500000e+04,17.81000,690.000000,12712.500000,36.000000,12.740000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,1.243221e+06,9.200000e+04,24.27000,710.000000,20000.000000,60.000000,15.990000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,1.646774e+06,1.500000e+06,999.00000,845.000000,40000.000000,60.000000,30.990000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


I'll drop the *id* variables now. Again, probably could have done this earlier.

In [218]:
lt = lt.drop(['id', 'member_id'], axis=1)

In [219]:
lt.head()

,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,loan_status_ind,grade_A,grade_B,grade_C,grade_D,home_ownership_MORTGAGE,home_ownership_RENT,purpose_credit_card,purpose_debt_consolidation,verification_status_Not Verified,verification_status_Verified,grade_E_below,home_ownership_not_mortgage_rent,purpose_other_reasons
0,44304.0,18.47,690.0,20000.0,60.0,17.93,0,0,0,0,0,1,0,0,1,0,1,1,0,0
1,136000.0,20.63,670.0,30000.0,36.0,11.99,1,0,0,1,0,1,0,0,1,0,1,0,0,0
2,50000.0,29.62,735.0,21500.0,36.0,11.99,1,0,1,0,0,0,1,0,1,0,1,0,0,0
3,64400.0,16.68,675.0,10000.0,36.0,13.67,1,0,0,1,0,0,1,0,1,0,1,0,0,0
4,88000.0,5.32,660.0,5000.0,36.0,8.49,1,0,1,0,0,1,0,0,1,0,1,0,0,0


I'm going to **make a change from the instructions** and **split my data now**. I want to do this correctly and split my data into the training and testing data frames. Then, I will standardize my training data. Finally, I will take the means and standard deviations from that standardization and use them on the testing data. 

In other words, no testing data is going to get used to standardize my training data. This is important - you don't want your testing values to influence the training data. If they get included in the standardization process, their observations will influence the means and standard deviations that are used to create the z-scores.

I'll split my data "by hand", rather than use anything from `sklearn`. If your data isn't randomly sorted, this would be a bad idea!

Also, `y` is not a data frame. I'll need to fix that before I can use it. `pandas` turned it into a **series** when I pulled out just a single column.

Finally, we are creating a logit **classification model**. You **do not** scale the target variable - keep it as a 1/0.

In [220]:
# Set y variable
y = lt['loan_status_ind']
y = y.to_frame(name='loan_status_ind')


# Set X variables
X = lt.drop('loan_status_ind', axis=1)

# Create training and test datasets
y_train = y[:20000]
X_train = X[:20000]
y_test = y[20000:]
X_test = X[20000:]

Now, I will standarize my X training data. Note how I use `fit_transform`. This will fit my standardization (i.e. calculate means and standard deviations) and then transform the data frame (i.e. turn everything into a z-score).

I'm also going to show you that you can **specifically name your scalers**. An "instance" of scaler is a scaler object. You can create one for you features, for you target if you're doing that, etc. You can create one that uses `StandardScaler()` and another that uses some other method.

When using AI generated code, it is still really helpful to know how Python works!

In [221]:
X_train

,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,grade_A,grade_B,grade_C,grade_D,home_ownership_MORTGAGE,home_ownership_RENT,purpose_credit_card,purpose_debt_consolidation,verification_status_Not Verified,verification_status_Verified,grade_E_below,home_ownership_not_mortgage_rent,purpose_other_reasons
0,44304.0,18.47,690.0,20000.0,60.0,17.93,0,0,0,0,1,0,0,1,0,1,1,0,0
1,136000.0,20.63,670.0,30000.0,36.0,11.99,0,0,1,0,1,0,0,1,0,1,0,0,0
2,50000.0,29.62,735.0,21500.0,36.0,11.99,0,1,0,0,0,1,0,1,0,1,0,0,0
3,64400.0,16.68,675.0,10000.0,36.0,13.67,0,0,1,0,0,1,0,1,0,1,0,0,0
4,88000.0,5.32,660.0,5000.0,36.0,8.49,0,1,0,0,1,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20002,124000.0,16.42,710.0,18000.0,36.0,11.55,0,1,0,0,1,0,0,1,0,1,0,0,0
20003,88000.0,24.49,695.0,5000.0,36.0,12.79,0,0,1,0,0,1,0,0,0,1,0,0,1
20004,43000.0,29.52,685.0,19750.0,60.0,15.99,0,0,0,1,0,1,1,0,1,0,0,0,0
20005,47000.0,15.60,730.0,14400.0,36.0,8.90,1,0,0,0,0,1,0,1,1,0,0,0,0


In [222]:
X_train.describe()

,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,grade_A,grade_B,grade_C,grade_D,home_ownership_MORTGAGE,home_ownership_RENT,purpose_credit_card,purpose_debt_consolidation,verification_status_Not Verified,verification_status_Verified,grade_E_below,home_ownership_not_mortgage_rent,purpose_other_reasons
count,2.000000e+04,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000
mean,7.721834e+04,18.590646,695.684500,14702.068750,42.825600,13.208164,0.16580,0.298650,0.293500,0.141300,0.497400,0.395800,0.219150,0.581900,0.314800,0.685200,0.100750,0.106800,0.19895
std,5.520197e+04,12.979611,31.292138,8788.919463,10.827347,4.733306,0.37191,0.457677,0.455377,0.348339,0.500006,0.489034,0.413681,0.493259,0.464448,0.464448,0.301005,0.308867,0.39922
min,2.000000e+02,0.000000,640.000000,600.000000,36.000000,5.320000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,4.650000e+04,12.090000,670.000000,8000.000000,36.000000,9.760000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,6.500000e+04,17.840000,690.000000,12800.000000,36.000000,12.740000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.00000
75%,9.200000e+04,24.290000,710.000000,20000.000000,60.000000,15.990000,0.00000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.00000
max,1.500000e+06,999.000000,845.000000,40000.000000,60.000000,30.990000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [223]:
# Create an instance of StandardScaler
X_scaler = StandardScaler()

# Standardize the columns in lt using z-scores
X_train = X_scaler.fit_transform(X_train)

# Convert the standardized array back to a DataFrame
X_train = pd.DataFrame(X_train, columns=X.columns)


In [224]:
X_train

,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,grade_A,grade_B,grade_C,grade_D,home_ownership_MORTGAGE,home_ownership_RENT,purpose_credit_card,purpose_debt_consolidation,verification_status_Not Verified,verification_status_Verified,grade_E_below,home_ownership_not_mortgage_rent,purpose_other_reasons
0,-0.596268,-0.009295,-0.181664,0.602812,1.586245,0.997602,-0.445818,-0.652550,-0.644537,-0.405649,1.005214,-0.809371,-0.529769,0.847649,-0.677811,0.677811,2.987567,-0.345789,-0.498359
1,1.064874,0.157124,-0.820818,1.740636,-0.630419,-0.257366,-0.445818,-0.652550,1.551501,-0.405649,1.005214,-0.809371,-0.529769,0.847649,-0.677811,0.677811,-0.334720,-0.345789,-0.498359
2,-0.493081,0.849766,1.256433,0.773485,-0.630419,-0.257366,-0.445818,1.532449,-0.644537,-0.405649,-0.994813,1.235528,-0.529769,0.847649,-0.677811,0.677811,-0.334720,-0.345789,-0.498359
3,-0.232214,-0.147207,-0.661029,-0.535013,-0.630419,0.097574,-0.445818,-0.652550,1.551501,-0.405649,-0.994813,1.235528,-0.529769,0.847649,-0.677811,0.677811,-0.334720,-0.345789,-0.498359
4,0.195318,-1.022448,-1.140395,-1.103925,-0.630419,-0.996826,-0.445818,1.532449,-0.644537,-0.405649,1.005214,-0.809371,-0.529769,0.847649,-0.677811,0.677811,-0.334720,-0.345789,-0.498359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.847485,-0.167239,0.457491,0.375247,-0.630419,-0.350327,-0.445818,1.532449,-0.644537,-0.405649,1.005214,-0.809371,-0.529769,0.847649,-0.677811,0.677811,-0.334720,-0.345789,-0.498359
19996,0.195318,0.454521,-0.021875,-1.103925,-0.630419,-0.088347,-0.445818,-0.652550,1.551501,-0.405649,-0.994813,1.235528,-0.529769,-1.179734,-0.677811,0.677811,-0.334720,-0.345789,2.006586
19997,-0.619891,0.842061,-0.341452,0.574366,1.586245,0.587730,-0.445818,-0.652550,-0.644537,2.465186,-0.994813,1.235528,1.887613,-1.179734,1.475337,-1.475337,-0.334720,-0.345789,-0.498359
19998,-0.547428,-0.230417,1.096645,-0.034370,-0.630419,-0.910203,2.243070,-0.652550,-0.644537,-0.405649,-0.994813,1.235528,-0.529769,0.847649,1.475337,-1.475337,-0.334720,-0.345789,-0.498359


Now, I'm going to **standardize the testing data using the fitted standardization from the training data**. To do this, I'm going to use `.transform` with the `X_scaler` object defined above.

In [225]:
# Standardize the columns in lt using z-scores
X_test = X_scaler.transform(X_test)

# Convert the standardized array back to a DataFrame
X_test = pd.DataFrame(X_test, columns=X.columns)

In [226]:
X_test

,annual_inc,dti,fico_range_low,loan_amnt,term,int_rate,grade_A,grade_B,grade_C,grade_D,home_ownership_MORTGAGE,home_ownership_RENT,purpose_credit_card,purpose_debt_consolidation,verification_status_Not Verified,verification_status_Verified,grade_E_below,home_ownership_not_mortgage_rent,purpose_other_reasons
0,-0.025694,-0.228105,-0.341452,-0.762578,-0.630419,-0.324974,-0.445818,1.532449,-0.644537,-0.405649,-0.994813,-0.809371,-0.529769,0.847649,1.475337,-1.475337,-0.33472,2.891936,-0.498359
1,-0.166997,-0.173403,-0.661029,0.147682,-0.630419,0.564490,-0.445818,-0.652550,1.551501,-0.405649,-0.994813,1.235528,-0.529769,0.847649,1.475337,-1.475337,-0.33472,-0.345789,-0.498359
2,-0.040187,0.135551,1.096645,-0.648795,-0.630419,-0.853159,-0.445818,1.532449,-0.644537,-0.405649,1.005214,-0.809371,-0.529769,-1.179734,-0.677811,0.677811,-0.33472,-0.345789,2.006586
3,-0.221344,0.550828,0.297702,0.983983,1.586245,0.444064,-0.445818,-0.652550,1.551501,-0.405649,1.005214,-0.809371,-0.529769,0.847649,-0.677811,0.677811,-0.33472,-0.345789,-0.498359
4,-0.138012,0.200269,-0.820818,-0.717065,-0.630419,0.799004,-0.445818,-0.652550,-0.644537,2.465186,-0.994813,1.235528,-0.529769,0.847649,-0.677811,0.677811,-0.33472,-0.345789,-0.498359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983,0.014160,-0.694233,-0.341452,-0.478122,-0.630419,-0.643998,-0.445818,1.532449,-0.644537,-0.405649,1.005214,-0.809371,-0.529769,-1.179734,1.475337,-1.475337,-0.33472,-0.345789,2.006586
4984,0.086623,-0.527043,-0.181664,-0.307448,-0.630419,0.237015,-0.445818,-0.652550,1.551501,-0.405649,-0.994813,1.235528,-0.529769,0.847649,1.475337,-1.475337,-0.33472,-0.345789,-0.498359
4985,-0.311923,-0.404540,-0.980606,1.171724,-0.630419,-0.109475,-0.445818,-0.652550,1.551501,-0.405649,1.005214,-0.809371,-0.529769,0.847649,-0.677811,0.677811,-0.33472,-0.345789,-0.498359
4986,-0.594529,-0.616416,-0.021875,-0.284692,1.586245,0.059545,-0.445818,-0.652550,1.551501,-0.405649,-0.994813,-0.809371,-0.529769,0.847649,1.475337,-1.475337,-0.33472,2.891936,-0.498359


Let me compare the y test and training data, just to see if they have similar means.

In [227]:
y_test.describe()

,loan_status_ind
count,4988.000000
mean,0.873897
std,0.331998
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [228]:
y_train.describe()

,loan_status_ind
count,20000.000000
mean,0.871350
std,0.334821
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


OK, let's use `smf.logit()` to estimate a model. This is **not** `sklearn`. I'm going to combine my y and X training data into one data frame and write out the "formula" for my logit model.

I'll try it with just annual income first.

In [229]:
# Concatenate the dependent variable and independent variables into a DataFrame
data = pd.concat([y_train, X_train[['annual_inc', 'dti', 'fico_range_low', 'loan_amnt', 'term', 'int_rate']]], axis=1)

# Define the formula for the logistic regression
formula = 'loan_status_ind ~ annual_inc'

# Fit the logistic regression model
model = smf.logit(formula=formula, data=data).fit()

# Print the summary of the model
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.383699
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:        loan_status_ind   No. Observations:                19993
Model:                          Logit   Df Residuals:                    19991
Method:                           MLE   Df Model:                            1
Date:                Sun, 21 Apr 2024   Pseudo R-squ.:               0.0005194
Time:                        23:07:04   Log-Likelihood:                -7671.3
converged:                       True   LL-Null:                       -7675.3
Covariance Type:            nonrobust   LLR p-value:                  0.004747
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.9140      0.021     90.496      0.000       1.873       1.955
annual_inc     0.0652      0.

In [230]:
# Define the formula for the logistic regression
formula = 'loan_status_ind ~ annual_inc + dti + fico_range_low + loan_amnt + term + int_rate'

# Fit the logistic regression model
model = smf.logit(formula=formula, data=data).fit()

# Print the summary of the model
print(model.summary())


Optimization terminated successfully.
         Current function value: 0.383632
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:        loan_status_ind   No. Observations:                19993
Model:                          Logit   Df Residuals:                    19986
Method:                           MLE   Df Model:                            6
Date:                Sun, 21 Apr 2024   Pseudo R-squ.:               0.0006932
Time:                        23:07:07   Log-Likelihood:                -7670.0
converged:                       True   LL-Null:                       -7675.3
Covariance Type:            nonrobust   LLR p-value:                    0.1002
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          1.9144      0.021     90.490      0.000       1.873       1.956
annual_inc       

Let me run the logit a few more times, adding new features to predict my target each time. I'll print my columns, so that I can copy and paste the variable names for my formula.

In [232]:
print(X_train.columns)

Index(['annual_inc', 'dti', 'fico_range_low', 'loan_amnt', 'term', 'int_rate',
       'grade_A', 'grade_B', 'grade_C', 'grade_D', 'home_ownership_MORTGAGE',
       'home_ownership_RENT', 'purpose_credit_card',
       'purpose_debt_consolidation', 'verification_status_Not Verified',
       'verification_status_Verified', 'grade_E_below',
       'home_ownership_not_mortgage_rent', 'purpose_other_reasons'],
      dtype='object')


I'm going to exclude *grade_A* from my estimate to avoid the **dummy variable trap**.

In [231]:
# Concatenate the dependent variable and independent variables into a DataFrame
data = pd.concat([y_train, X_train[['annual_inc', 'dti', 'fico_range_low', 'loan_amnt', 'term', 'int_rate', 'grade_A', 'grade_B', 'grade_C', 'grade_D',
       'grade_E_below']]], axis=1)

# Define the formula for the logistic regression
formula = 'loan_status_ind ~ annual_inc + dti + fico_range_low + loan_amnt + term + int_rate + grade_B + grade_C + grade_D + grade_E_below'

# Fit the logistic regression model
model = smf.logit(formula=formula, data=data).fit()

# Print the summary of the model
print(model.summary())


Optimization terminated successfully.
         Current function value: 0.383589
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:        loan_status_ind   No. Observations:                19993
Model:                          Logit   Df Residuals:                    19982
Method:                           MLE   Df Model:                           10
Date:                Sun, 21 Apr 2024   Pseudo R-squ.:               0.0008068
Time:                        23:07:14   Log-Likelihood:                -7669.1
converged:                       True   LL-Null:                       -7675.3
Covariance Type:            nonrobust   LLR p-value:                    0.2602
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          1.9146      0.021     90.481      0.000       1.873       1.956
annual_inc       

Not a very inspring model at this point.

I am **not going to include the "other" indicators** in my model. This is avoid the **dummy variable trap**. So, for example, the coefficients for *home_ownership_MORTGAGE* and *home_ownership_RENT* are estimated **relative to all of the other categories that I grouped together and left out of the model. This is really only important if you're going to try to interpret the coefficients.

If I didn't combine my indicators this way, my logit would have a hard time estimating. For example, I would need to increase the number of iterations. I show you how you can do that below - it's not necessary, though.

In [234]:
# Concatenate the dependent variable and independent variables into a DataFrame
data = pd.concat([y_train, X_train[['annual_inc', 'dti', 'fico_range_low', 'loan_amnt', 'term', 'int_rate', 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E_below',
       'home_ownership_MORTGAGE', 'home_ownership_RENT', 'home_ownership_not_mortgage_rent',
       'purpose_credit_card', 'purpose_debt_consolidation', 'purpose_other_reasons',
       'verification_status_Not Verified', 'verification_status_Verified']]], axis=1)

# Define the formula for the logistic regression
formula = 'loan_status_ind ~ annual_inc + dti + fico_range_low + loan_amnt + term + int_rate + \
            grade_B + grade_C + grade_D + grade_E_below + home_ownership_MORTGAGE + home_ownership_RENT  + \
            purpose_credit_card + purpose_debt_consolidation + \
            verification_status_Verified'

# Fit the logistic regression model
model = smf.logit(formula=formula, data=data).fit(maxiter=1000)

# Print the summary of the model
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.383335
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:        loan_status_ind   No. Observations:                19993
Model:                          Logit   Df Residuals:                    19977
Method:                           MLE   Df Model:                           15
Date:                Sun, 21 Apr 2024   Pseudo R-squ.:                0.001468
Time:                        23:10:43   Log-Likelihood:                -7664.0
converged:                       True   LL-Null:                       -7675.3
Covariance Type:            nonrobust   LLR p-value:                   0.09439
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        1.9164      0.021     90.423      0.000

Now, let's turn to `sklearn`. I'm going to start by just keeping some of the features I want in a new data frame. 

In [235]:
columns_to_keep = ['annual_inc', 'dti', 'fico_range_low', 'loan_amnt', 'term', 'int_rate', 'grade_B', 'grade_C', 'grade_D',
       'grade_E_below']
X_train_exclusions = X_train.loc[:, columns_to_keep]

In [236]:
#Create an instance of logisticregression named lgstc_reg 
lgstc_reg =  LogisticRegression(penalty = None, solver="newton-cg", max_iter = 5000)     

# Fit logististic regression to training set

lgstc_reg.fit(X_train_exclusions, y_train)                                       
print(lgstc_reg.intercept_, lgstc_reg.coef_)

# Calculate the score on the training data
train_score = lgstc_reg.score(X_train_exclusions, y_train)
print(f"Training accuracy: {train_score:.4f}")
 

[2.07541848] [[ 0.12443085 -0.03282605  0.15933503 -0.06767981  0.01737678  0.10906872
  -0.27162196 -0.53800304 -0.57707766 -0.67734214]]
Training accuracy: 0.8712


/home/codespace/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


We can try even more variables.

In [250]:
columns_to_keep = ['annual_inc', 'dti', 'fico_range_low', 'loan_amnt', 'term', 'int_rate', 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E_below',
       'home_ownership_MORTGAGE', 'home_ownership_RENT',
       'purpose_credit_card', 'purpose_debt_consolidation',
       'verification_status_Verified']
X_train_exclusions = X_train.loc[:, columns_to_keep]

In [251]:
#Create an instance of logisticregression named lgstc_reg 
lgstc_reg =  LogisticRegression(penalty = None, solver="sag", max_iter = 5000)     

# Fit logististic regression to training set

lgstc_reg.fit(X_train_exclusions, y_train)                                       
print(lgstc_reg.intercept_, lgstc_reg.coef_) 

# Calculate the score on the training data
train_score = lgstc_reg.score(X_train_exclusions, y_train)
print(f"Training accuracy: {train_score:.4f}")

/home/codespace/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[2.08066166] [[ 0.10895374 -0.03491904  0.15136372 -0.06319647  0.00666339  0.12493686
   0.3657222   0.18702585 -0.07460208 -0.21978353 -0.36903724  0.01758518
  -0.0680922   0.04101706 -0.03421567 -0.06297665]]
Training accuracy: 0.8712


You can print out the coefficients in a nicer format too.

In [252]:
# Print the coefficients
coefficients = pd.DataFrame({'Variable': X_train_exclusions.columns, 'Coefficient': lgstc_reg.coef_[0]})
print(coefficients)

                        Variable  Coefficient
0                     annual_inc     0.108954
1                            dti    -0.034919
2                 fico_range_low     0.151364
3                      loan_amnt    -0.063196
4                           term     0.006663
5                       int_rate     0.124937
6                        grade_A     0.365722
7                        grade_B     0.187026
8                        grade_C    -0.074602
9                        grade_D    -0.219784
10                 grade_E_below    -0.369037
11       home_ownership_MORTGAGE     0.017585
12           home_ownership_RENT    -0.068092
13           purpose_credit_card     0.041017
14    purpose_debt_consolidation    -0.034216
15  verification_status_Verified    -0.062977


I like to look and see if the signs on my coefficients make any sense whatsoever. 

Higher annual income, high low range of FICO score, higher Grade score -> more likely to be a performing loan

Renting is negative, which makes sense. Could be a proxy for age, though. Need to be careful here - it is very much illegal to discriminate based on what the law calls a **protected class**. 

<https://www.law.cornell.edu/wex/protected_class>

These are very real issues as data gets used to make more decisions. <https://www.nytimes.com/2020/09/18/business/digital-mortgages.html>

**Chapter 11 of Hull** discusses these issues in more detail.

Let's try some prediction. We will predict our target variable using the X training features. The logit model outputs two columns for predictions. The first column is the probability of being a zero. The second column is the probaility of being a 1. Obviously, the second column is just 1 - the first column. We'll keep the second column.

In [254]:
y_train_pred = lgstc_reg.predict_proba(X_train_exclusions)
y_train_pred

array([[0.30690457, 0.69309543],
       [0.15292876, 0.84707124],
       [0.0918302 , 0.9081698 ],
       ...,
       [0.19254998, 0.80745002],
       [0.04838713, 0.95161287],
       [0.13756475, 0.86243525]])

In [255]:
y_train_pred = y_train_pred[:,1]
y_train_pred[:5]

array([0.69309543, 0.84707124, 0.9081698 , 0.83466392, 0.90634051])

I printed out the first five predicted probabilities above. 

We need to turn these predicted probabilities to either a 1 or a 0. We need to turn them into **binary predictions** to compare to the actual values. 

To do that, we need a **threshold**. How large does the probability need to be in order for us to say, yes, that observation is a 1?

We can use a **confusion matrix** to took at the accuracy of our model using the training data. Does it capture true negatives (i.e. a predicted 0 is a real 0)? How about true positives (i.e. a predicted 1 is a true 1)? How many false negatives and false positives does our model give, using that particular theshold?

In [256]:
# Set a threshold to binarize the predicted probabilities
threshold = 0.80
y_pred_binary = np.where(y_train_pred >= threshold, 1, 0)
y_pred_binary[:5]

array([0, 1, 1, 1, 1])

In [257]:
# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_train, y_pred_binary).ravel()

# Print the confusion matrix
print("Confusion Matrix:")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives: {tp}")

Confusion Matrix:
True Negatives: 862
False Positives: 1711
False Negatives: 2397
True Positives: 15030


Finally, let's take the **fitted model** and use our **X_test** features to predict some values. This is our **out-of-sample** test.

Does the threshold we picked above using the training data work with our test data?

In [258]:
X_test_exclusions = X_test.loc[:, columns_to_keep]

In [259]:
y_test_pred = lgstc_reg.predict_proba(X_test_exclusions)
y_test_pred = y_test_pred[:,1]
y_test_pred[:5]

array([0.92425788, 0.85543221, 0.93091404, 0.86731826, 0.7729421 ])

In [260]:
# Set a threshold to binarize the predicted probabilities
threshold = 0.80
y_pred_binary = np.where(y_test_pred >= threshold, 1, 0)
y_pred_binary[:5]

array([1, 1, 1, 1, 0])

In [261]:
mean = np.mean(y_pred_binary)
print("Mean:", mean)

Mean: 0.8358059342421812


In [262]:
mean = np.mean(y_test)
print("Mean:", mean)

Mean: 0.873897353648757


In [263]:
# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()

# Print the confusion matrix
print("Confusion Matrix:")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives: {tp}")

Confusion Matrix:
True Negatives: 225
False Positives: 404
False Negatives: 594
True Positives: 3765


Not great! You can try to fine-tune the prediction by searching around for a better threshold in the training data. 

## Part 3

Finally, **clear the outputs of all of your cells** using the button at the top of the notebook. Restart your Python kernel. This will clear everything from memory. Then, **Run All**. All of you cells should now run, in order, from top to bottom.

I like to do this occasionally, to make sure things are working correctly. One downside of working in a notebook environment is that it is very easy to run code "out of order". For example, you can run code in one cell, then skip down and run another cell. But, if that cell below needed something from a cell in the middle to run, it won't work. You can lose track of what variables have been defined, what inputs have been created, what Python "knows" about. So, it's a good idea to run things from scratch.

You are done! Make sure that you've done the git add/commit/push steps.